In [ ]:
!python -m pip install --upgrade pip
!python -m pip install accelerate>=0.20.1 -U
!python -m pip install transformers[torch] -U
!pip install datasets
!pip install jiwer
import os

os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [1]:
import nltk
import re
import json
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC
nltk.download('stopwords')
import os
from sklearn.model_selection import train_test_split
from transformers import Trainer
import pandas as pd
import zipfile
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor
import soundfile as sf
from tqdm import tqdm
import IPython.display as ipd
import numpy as np
import random
import torch
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
from datasets import load_metric
from transformers import TrainingArguments
from IPython.display import display, HTML

/Volumes/ExternalSSD/ESIR3/AI-S9/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yannpaillard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'soundfile'

# Projet

## ======= Charger les données =======
Nous allons tout d'abord importer les données.

In [2]:
onDrive = True

if onDrive:
    from google.colab import drive

    drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cp /content/drive/MyDrive/AI/Data/Common\ Language.zip /content/

In [4]:
def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)


if onDrive:
    unzip_file("/content/Common Language.zip", "/content/")
else:
    unzip_file("./Data/Common Language.zip", "./")

In [109]:
# Chemin du dossier principal
if onDrive:
    base_path = "/content/common_voice_kpd"
else:
    base_path = "./common_voice_kpd"

# Initialiser des dataframes vides
train_df = pd.DataFrame()
test_df = pd.DataFrame()
dev_df = pd.DataFrame()

languages_to_include = ['English', 'French']

if onDrive:
    for language_folder in os.listdir(base_path):
        if language_folder in languages_to_include:
            folder_path = os.path.join(base_path, language_folder)
            if os.path.isdir(folder_path):
                # Chemin des fichiers
                train_file = os.path.join(folder_path, 'train.csv')
                test_file = os.path.join(folder_path, 'test.csv')
                dev_file = os.path.join(folder_path, 'dev.csv')

                # Lire et ajouter la colonne de langue, mettre à jour la colonne 'path'
                if os.path.exists(train_file):
                    temp_df = pd.read_csv(train_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/train/' + temp_df['client_id'] + "/" + temp_df[
                        'path']  # Mettre à jour la colonne 'path'
                    train_df = pd.concat([train_df, temp_df])

                if os.path.exists(test_file):
                    temp_df = pd.read_csv(test_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/test/' + temp_df['client_id'] + "/" + temp_df['path']
                    test_df = pd.concat([test_df, temp_df])

                if os.path.exists(dev_file):
                    temp_df = pd.read_csv(dev_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/dev/' + temp_df['client_id'] + "/" + temp_df['path']
                    dev_df = pd.concat([dev_df, temp_df])
else:
    for language_folder in os.listdir(base_path):
        if language_folder in languages_to_include:
            folder_path = os.path.join(base_path, language_folder)
            if os.path.isdir(folder_path):
                # Chemin des fichiers
                train_file = os.path.join(folder_path, 'train.csv')
                test_file = os.path.join(folder_path, 'test.csv')
                dev_file = os.path.join(folder_path, 'dev.csv')

                # Lire et ajouter la colonne de langue, mettre à jour la colonne 'path'
                if os.path.exists(train_file):
                    temp_df = pd.read_csv(train_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/train/' + temp_df['client_id'] + "/" + temp_df[
                        'path']  # Mettre à jour la colonne 'path'
                    train_df = pd.concat([train_df, temp_df])

                if os.path.exists(test_file):
                    temp_df = pd.read_csv(test_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/test/' + temp_df['client_id'] + "/" + temp_df['path']
                    test_df = pd.concat([test_df, temp_df])

                if os.path.exists(dev_file):
                    temp_df = pd.read_csv(dev_file, sep='\t', encoding='utf-16')
                    temp_df['language'] = language_folder
                    temp_df['path'] = folder_path + '/dev/' + temp_df['client_id'] + "/" + temp_df['path']
                    dev_df = pd.concat([dev_df, temp_df])

if not onDrive:
    # replace all \ with / in path column
    train_df['path'] = train_df['path'].str.replace('\\', '/')
    test_df['path'] = test_df['path'].str.replace('\\', '/')
    dev_df['path'] = dev_df['path'].str.replace('\\', '/')

# Suppression de colonnes inutiles
columns_to_remove = ["client_id", "age", "gender", "Unnamed: 0"]
train_df = train_df.drop(columns=columns_to_remove, errors='ignore')
test_df = test_df.drop(columns=columns_to_remove, errors='ignore')
dev_df = dev_df.drop(columns=columns_to_remove, errors='ignore')

# Réinitialiser les index des dataframes
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)

concatenated_df = pd.concat([train_df, test_df, dev_df], ignore_index=True)

# Split the concatenated dataframe into train and test sets
train_df, test_df = train_test_split(concatenated_df, test_size=0.40)

# Reset index for both train and test sets
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [110]:
train_df

,path,sentence,language
0,/content/common_voice_kpd/French/train/frn_trn...,Ces livres montrent trois différents futurs du...,French
1,/content/common_voice_kpd/English/train/eng_tr...,"Further, there is no mention of the comet in C...",English
2,/content/common_voice_kpd/English/dev/eng_dev_...,They had gone away to breakfast at Henderson's...,English
3,/content/common_voice_kpd/English/dev/eng_dev_...,This knowledge is delivered in summaries of th...,English
4,/content/common_voice_kpd/French/train/frn_trn...,On doit cette appellation à l'oiseau représent...,French
...,...,...,...
722,/content/common_voice_kpd/English/train/eng_tr...,Maybe no one here knows what an alchemist is!,English
723,/content/common_voice_kpd/English/train/eng_tr...,Thanks for dropping around.,English
724,/content/common_voice_kpd/French/test/frn_tst_...,Le pays est ainsi découpé en plusieurs circons...,French
725,/content/common_voice_kpd/English/train/eng_tr...,One picture in the gallery shows how diligent ...,English


In [111]:
test_df

,path,sentence,language
0,/content/common_voice_kpd/English/train/eng_tr...,Every day was there to be lived or to mark one...,English
1,/content/common_voice_kpd/French/test/frn_tst_...,Sa langue maternelle est le hongrois.,French
2,/content/common_voice_kpd/French/train/frn_trn...,Ainsi avorta ce fœtus de crime enfanté par Bru...,French
3,/content/common_voice_kpd/French/train/frn_trn...,La commune de Saint-Étienne-des-Oullières se t...,French
4,/content/common_voice_kpd/French/train/frn_trn...,Elle indique le nord et le nom des victimes y ...,French
...,...,...,...
481,/content/common_voice_kpd/English/train/eng_tr...,Some storms are worth the wreckage.,English
482,/content/common_voice_kpd/English/dev/eng_dev_...,"By accepting our privacy policy, you agree tha...",English
483,/content/common_voice_kpd/English/train/eng_tr...,It was later revealed that the letter was a pr...,English
484,/content/common_voice_kpd/French/train/frn_trn...,un rue Jacques Barraja,French


## ======= première analyse des données =======

nous cherchons d'abord la taille de l'ensemble de données

In [112]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"”_]'

def remove_special_characters(text):

    # Remove all characters that are not letters or numbers
    text = re.sub(r'[^a-zA-Z0-9 éè]', '', text)

    # Convert to lowercase
    text = text.lower()

    return text

In [113]:
train_df['sentence'] = train_df['sentence'].apply(remove_special_characters)
test_df['sentence'] = test_df['sentence'].apply(remove_special_characters)

In [114]:
def extract_all_chars(df):
    all_text = ' '.join(df['sentence'])
    vocab = list(set(all_text))
    return {"vocab": vocab, "all_text": all_text}


# Appliquer la fonction sur l'ensemble du DataFrame
vocabs_train = extract_all_chars(train_df)
vocabs_test = extract_all_chars(test_df)

In [115]:
vocab_list = list(set(vocabs_train['vocab']) | set(vocabs_test['vocab']))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'f': 0,
 'q': 1,
 'p': 2,
 'r': 3,
 'k': 4,
 'l': 5,
 'z': 6,
 's': 7,
 'h': 8,
 'y': 9,
 'c': 10,
 'n': 11,
 'w': 12,
 'a': 13,
 'x': 14,
 ' ': 15,
 'j': 16,
 'b': 17,
 't': 18,
 'è': 19,
 'o': 20,
 'e': 21,
 'g': 22,
 'm': 23,
 'u': 24,
 'd': 25,
 'i': 26,
 'é': 27,
 'v': 28}

In [116]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [117]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))
vocab_dict

31


{'f': 0,
 'q': 1,
 'p': 2,
 'r': 3,
 'k': 4,
 'l': 5,
 'z': 6,
 's': 7,
 'h': 8,
 'y': 9,
 'c': 10,
 'n': 11,
 'w': 12,
 'a': 13,
 'x': 14,
 'j': 16,
 'b': 17,
 't': 18,
 'è': 19,
 'o': 20,
 'e': 21,
 'g': 22,
 'm': 23,
 'u': 24,
 'd': 25,
 'i': 26,
 'é': 27,
 'v': 28,
 '|': 15,
 '[UNK]': 29,
 '[PAD]': 30}

In [118]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [119]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [120]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True,
                                             return_attention_mask=False)

In [121]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [122]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch


# Appliquer la fonction sur chaque ligne du DataFrame
tqdm.pandas(desc="Processing1")
train_df = train_df.progress_apply(speech_file_to_array_fn, axis=1)
tqdm.pandas(desc="Processing2")
test_df = test_df.progress_apply(speech_file_to_array_fn, axis=1)

# Supprimer les colonnes spécifiques
columns_to_remove = ['path', 'sentence']  # Mettez ici les noms des colonnes à enlever
train_df = train_df.drop(columns=columns_to_remove)
test_df = test_df.drop(columns=columns_to_remove)



Processing2: 100%|██████████| 486/486 [00:01<00:00, 473.08it/s]


In [123]:
# Sélectionner un index aléatoire
rand_int = random.randint(0, len(train_df) - 1)

# Lire les données audio de l'échantillon sélectionné
audio_data = train_df.iloc[rand_int]["speech"]
sampling_rate = train_df.iloc[rand_int]["sampling_rate"]  # Assurez-vous d'avoir cette colonne dans votre DataFrame

# Jouer l'audio
ipd.Audio(data=np.asarray(audio_data), autoplay=True, rate=sampling_rate)

In [124]:
# Selectionner un index aléatoire
rand_int = random.randint(0, len(test_df) - 1)

# Lire les données audio de l'échantillon sélectionné
audio_data = test_df.iloc[rand_int]["speech"]
sampling_rate = test_df.iloc[rand_int]["sampling_rate"]  # Assurez-vous d'avoir cette colonne dans votre DataFrame

# Jouer l'audio
ipd.Audio(data=np.asarray(audio_data), autoplay=True, rate=sampling_rate)

In [125]:
def prepare_dataset(row):
    # Assurer que l'échantillon a le bon taux d'échantillonnage
    assert row[
               "sampling_rate"] == processor.feature_extractor.sampling_rate, f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    # Traiter les données audio
    input_values = processor(row["speech"], sampling_rate=row["sampling_rate"]).input_values

    # Traiter les étiquettes
    labels = processor(text=row["target_text"]).input_ids

    row["input_values"] = input_values
    row["labels"] = labels
    return row


tqdm.pandas(desc="Processing")
# Appliquer la fonction sur chaque ligne du DataFrame
train_df_prepared = train_df.progress_apply(prepare_dataset, axis=1)
test_df_prepared = test_df.progress_apply(prepare_dataset, axis=1)

# Supprimer les colonnes inutiles
columns_to_remove = []
train_df_prepared = train_df_prepared.drop(columns=columns_to_remove)
test_df_prepared = test_df_prepared.drop(columns=columns_to_remove)


Processing: 100%|██████████| 486/486 [00:00<00:00, 489.46it/s]


In [126]:
train_df_prepared

,language,speech,sampling_rate,target_text,input_values,labels
0,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,ces livres montrent trois différents futurs du...,"[[-0.00056577666, -0.00056577666, -0.000565776...","[10, 21, 7, 15, 5, 26, 28, 3, 21, 7, 15, 23, 2..."
1,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,further there is no mention of the comet in ch...,"[[0.0012011413, 0.0012011413, 0.0012011413, 0....","[0, 24, 3, 18, 8, 21, 3, 15, 18, 8, 21, 3, 21,..."
2,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,they had gone away to breakfast at hendersons ...,"[[-0.00020431612, -0.00020431612, -0.000204316...","[18, 8, 21, 9, 15, 8, 13, 25, 15, 22, 20, 11, ..."
3,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,this knowledge is delivered in summaries of th...,"[[0.001021908, 0.001021908, 0.001021908, 0.001...","[18, 8, 26, 7, 15, 4, 11, 20, 12, 5, 21, 25, 2..."
4,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,on doit cette appellation loiseau représenté ...,"[[0.00067544373, 0.00067544373, 0.00067544373,...","[20, 11, 15, 25, 20, 26, 18, 15, 10, 21, 18, 1..."
...,...,...,...,...,...,...
722,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,maybe no one here knows what an alchemist is,"[[-0.0015673003, -0.0015673003, -0.0015673003,...","[23, 13, 9, 17, 21, 15, 11, 20, 15, 20, 11, 21..."
723,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,thanks for dropping around,"[[0.000734832, 0.000734832, 0.000734832, 0.000...","[18, 8, 13, 11, 4, 7, 15, 0, 20, 3, 15, 25, 3,..."
724,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,le pays est ainsi découpé en plusieurs circons...,"[[-0.0024077604, -0.0024077604, -0.0024077604,...","[5, 21, 15, 2, 13, 9, 7, 15, 21, 7, 18, 15, 13..."
725,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,one picture in the gallery shows how diligent ...,"[[0.00013209238, 0.00013209238, 0.00013209238,...","[20, 11, 21, 15, 2, 26, 10, 18, 24, 3, 21, 15,..."


In [127]:
test_df_prepared

,language,speech,sampling_rate,target_text,input_values,labels
0,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,every day was there to be lived or to mark one...,"[[-0.00019109882, -0.00019109882, -0.000191098...","[21, 28, 21, 3, 9, 15, 25, 13, 9, 15, 12, 13, ..."
1,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,sa langue maternelle est le hongrois,"[[2.9233737e-05, 2.9233737e-05, 2.9233737e-05,...","[7, 13, 15, 5, 13, 11, 22, 24, 21, 15, 23, 13,..."
2,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,ainsi avorta ce ftus de crime enfanté par bruj...,"[[0.0010821358, 0.0010821358, 0.0010821358, 0....","[13, 26, 11, 7, 26, 15, 13, 28, 20, 3, 18, 13,..."
3,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,la commune de sainttiennedesoullières se trouv...,"[[-0.002408421, -0.002408421, -0.002408421, -0...","[5, 13, 15, 10, 20, 23, 23, 24, 11, 21, 15, 25..."
4,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,elle indique le nord et le nom des victimes y ...,"[[8.459496e-06, 8.459496e-06, 8.459496e-06, 8....","[21, 5, 5, 21, 15, 26, 11, 25, 26, 1, 24, 21, ..."
...,...,...,...,...,...,...
481,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,some storms are worth the wreckage,"[[-0.004857998, -0.004857998, -0.004857998, -0...","[7, 20, 23, 21, 15, 7, 18, 20, 3, 23, 7, 15, 1..."
482,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,by accepting our privacy policy you agree that...,"[[0.0003696216, 0.0003696216, 0.0003696216, 0....","[17, 9, 15, 13, 10, 10, 21, 2, 18, 26, 11, 22,..."
483,English,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,it was later revealed that the letter was a pr...,"[[0.00047213104, 0.00047213104, 0.00047213104,...","[26, 18, 15, 12, 13, 7, 15, 5, 13, 18, 21, 3, ..."
484,French,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,un rue jacques barraja,"[[0.0006915888, 0.0006915888, 0.0006915888, 0....","[24, 11, 15, 3, 24, 21, 15, 16, 13, 10, 1, 24,..."


In [128]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = []
        for feature in features:
            input_values = feature["input_values"]
            if len(input_values) == 1 and isinstance(input_values[0], np.ndarray):
                # If input_values is a list of a single numpy.ndarray, flatten it
                input_features.append({"input_values": input_values[0]})
            else:
                input_features.append({"input_values": input_values})

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )


        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
        )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [129]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [130]:
wer_metric = load_metric("wer")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [131]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [132]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xlsr-53-espeak-cv-ft",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)
model.gradient_checkpointing_enable()
print("Model's pad_token_id:", model.config.pad_token_id)

Some weights of the model checkpoint at facebook/wav2vec2-xlsr-53-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xlsr-53-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should prob

Model's pad_token_id: 30


In [133]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1919: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [134]:
training_args = TrainingArguments(
    output_dir="./saves",
    # output_dir="./wav2vec2-base-timit-demo",
    group_by_length=True,
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=200,
    save_total_limit=1,
)

In [135]:
train_dataset = train_df_prepared.to_dict(orient='records')
eval_dataset = test_df_prepared.to_dict(orient='records')

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)


In [136]:
"""
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
"""

'\nfunction ConnectButton(){\n    console.log("Connect pushed");\n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()\n}\nsetInterval(ConnectButton,60000);\n'

In [137]:
! export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:8
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,3.853700,4.309486,1.000000
200,3.129600,3.121052,1.000000
300,3.058000,3.075946,1.000000
400,2.538800,2.322765,0.994787
500,1.658100,1.633842,0.879420
600,1.425700,1.442554,0.786491
700,1.244500,1.355570,0.744787
800,1.160300,1.301453,0.726201
900,1.181800,1.399539,0.700589
1000,0.888400,1.252772,0.678377


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

TrainOutput(global_step=2730, training_loss=2.187749629142957, metrics={'train_runtime': 1255.6563, 'train_samples_per_second': 17.369, 'train_steps_per_second': 2.174, 'total_flos': 6.281336971607409e+18, 'train_loss': 2.187749629142957, 'epoch': 30.0})

In [171]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [172]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")

Some weights of the model checkpoint at patrickvonplaten/wav2vec2-base-timit-demo were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-timit-demo and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You shoul

In [173]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"],
      sampling_rate=batch["sampling_rate"],
      return_tensors="pt"
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]

  return batch

In [174]:
results = test_df_prepared.apply(map_to_result, axis=1)

In [175]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.903


In [176]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = random.sample(range(len(dataset)), num_examples)

    df = dataset.iloc[picks].drop(columns=["speech", "sampling_rate", "input_values", "labels"])
    display(HTML(df.to_html()))

show_random_elements(results)

,language,target_text,pred_str
44,French,celui du royaumeuni prne une durée de travail plus longue,sureread your youiet poison in juiflinta tyvape preuon
302,English,the longest way round is the sweetest way home,the longest raroum is the sweetest wey home
216,French,mais je citerai un seul exemple,mesuris eaty outcitde exump
16,French,lun et lautre de ces corps commencent sorganiser assez bien,herlong amotal dicsypqo cermo's er sok un easy as she d
15,French,ils sont représentés sur les temples hindous partir de lépoque médiévale,it's all over he onis wal wil le alhar at weltu elativ vake yet
47,English,at that time the austrians were the reputed masters of light cavalry work,at that time the ostrieans with a reputed mosters of light coverely welk
111,French,quatre,carm
119,French,pour les inconditionnels du modèle économique des coopératives cétait inacceptable,wovalesim osis shune tu mardan equldmamec dequicane site enexet
258,English,compare the checksum to verify its integrity,you're pay the chek some to varify hisintagrati
308,English,its motto on the nameplate below the title is one of americas great newspapers,e mouciavl are the vier bedo i vu e one of lamicals creamel p


In [177]:
model.to("cuda")
input_values = processor(test_df_prepared.iloc[0]["speech"], sampling_rate=test_df_prepared.iloc[0]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
  logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] h i [PAD] s s | | f f [PAD] o o o r r m [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] e [PAD] [PAD] [PAD] v e e [PAD] r r y y | | d [PAD] e e a [PAD] [PAD] v v e e r | | i s s s | | t h e a r r e | | | t o o | | | m [PAD] u u [PAD] [PAD] [PAD] [PAD] [PAD] l l [PAD] i i i [PAD] [PAD] v v e e [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | | [PA

In [178]:
results_test = []
model.to("cuda")
for index, row in test_df_prepared.iterrows():
    # Process the input
    input_values = processor(row["speech"], sampling_rate=row["sampling_rate"], return_tensors="pt").input_values.to("cuda")

    with torch.no_grad():
        # Pass both the input_values (on CUDA) and their lengths (on CPU) to the model
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)

    results_test.append((processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()), row["language"]))


results_train = []
model.to("cuda")
for index, row in train_df_prepared.iterrows():
    # Process the input
    input_values = processor(row["speech"], sampling_rate=row["sampling_rate"], return_tensors="pt").input_values.to("cuda")

    with torch.no_grad():
        # Pass both the input_values (on CUDA) and their lengths (on CPU) to the model
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)

    results_train.append((processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()), row["language"]))


In [179]:
# Convert the 'results' list to a DataFrame
df_results_test = pd.DataFrame(results_test, columns=['phoneme', 'language'])

# Display the first few rows of the DataFrame to verify
df_results_test

,phoneme,language
0,"[[PAD], [PAD], h, i, [PAD], s, s, |, |, f, f, ...",English
1,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
2,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
3,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
4,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
...,...,...
481,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
482,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
483,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
484,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French


In [180]:
# Convert the 'results' list to a DataFrame
df_results_train = pd.DataFrame(results_train, columns=['phoneme', 'language'])

# Display the first few rows of the DataFrame to verify
df_results_train

,phoneme,language
0,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
1,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
2,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
3,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
4,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
...,...,...
722,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
723,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English
724,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",French
725,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",English


In [181]:
# Path to the CSV file
csv_file_path = 'merged_data.csv'

# Merge the DataFrames and shuffle
df_merged = pd.concat([df_results_test, df_results_train], ignore_index=True)
df_merged = df_merged.sample(frac=1).reset_index(drop=True)

# Optionally, save the merged DataFrame to a CSV file for future use
df_merged.to_csv(csv_file_path, index=False)
print("Saved merged DataFrame to CSV.")

Saved merged DataFrame to CSV.
Training set size: 970, Testing set size: 243
